In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train_data = pd.read_csv("train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_data = pd.read_csv("test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

% of women who survived: 0.7420382165605095


In [5]:
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

% of men who survived: 0.18890814558058924


In [6]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

# Preprocessing
train_data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
test_data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

# Handle missing values in 'Age' and 'Embarked' columns
train_data['Age'].fillna(train_data['Age'].median(), inplace=True)
train_data['Embarked'].fillna(train_data['Embarked'].mode()[0], inplace=True)
test_data['Age'].fillna(test_data['Age'].median(), inplace=True)
test_data['Fare'].fillna(test_data['Fare'].median(), inplace=True)

# Convert 'Sex' and 'Embarked' columns to numerical values
train_data.replace({'Sex': {'male': 0, 'female': 1}, 'Embarked': {'S': 0, 'C': 1, 'Q': 2}}, inplace=True)
test_data.replace({'Sex': {'male': 0, 'female': 1}, 'Embarked': {'S': 0, 'C': 1, 'Q': 2}}, inplace=True)

# Standardize the numerical features
scaler = StandardScaler()
X_train = scaler.fit_transform(train_data.drop('Survived', axis=1))
y_train = train_data['Survived'].values.reshape(-1, 1)
X_test = scaler.transform(test_data)

# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=64, activation='relu', input_shape=[X_train.shape[1]]),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32)

# Make predictions on the test set
predictions = model.predict(X_test)
predictions = np.round(predictions).astype(int).flatten()

# Create the submission file
submission = pd.DataFrame({'PassengerId': test_data.index + 892, 'Survived': predictions})
submission.to_csv('submission.csv', index=False)

print("Submission file created successfully!")

2023-04-28 21:09:06.475893: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/50
28/28 [==============================] - 1s 2ms/step - loss: 0.6285 - accuracy: 0.7217
Epoch 2/50
28/28 [==============================] - 0s 2ms/step - loss: 0.4988 - accuracy: 0.8092
Epoch 3/50
28/28 [==============================] - 0s 2ms/step - loss: 0.4426 - accuracy: 0.8114
Epoch 4/50
28/28 [==============================] - 0s 2ms/step - loss: 0.4235 - accuracy: 0.8182
Epoch 5/50
28/28 [==============================] - 0s 2ms/step - loss: 0.4139 - accuracy: 0.8227
Epoch 6/50
28/28 [==============================] - 0s 2ms/step - loss: 0.4096 - accuracy: 0.8204
Epoch 7/50
28/28 [==============================] - 0s 2ms/step - loss: 0.4055 - accuracy: 0.8350
Epoch 8/50
28/28 [==============================] - 0s 2ms/step - loss: 0.4006 - accuracy: 0.8373
Epoch 9/50
28/28 [==============================] - 0s 2ms/step - loss: 0.3983 - accuracy: 0.8406
Epoch 10/50
28/28 [==============================] - 0s 1ms/step - loss: 0.3951 - accuracy: 0.8429
Epoch 11/50
28/28 [

In [7]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load actual survival values from gender_submission.csv
actual_data = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')
y_actual = actual_data['Survived'].values

# Load predicted survival values from submission.csv
predicted_data = pd.read_csv('/kaggle/working/submission.csv')
y_predicted = predicted_data['Survived'].values

# Calculate evaluation metrics
accuracy = accuracy_score(y_actual, y_predicted)
precision = precision_score(y_actual, y_predicted)
recall = recall_score(y_actual, y_predicted)
f1 = f1_score(y_actual, y_predicted)

# Print evaluation metrics
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/titanic/gender_submission.csv'